In [17]:
%pip install earthengine-api
%pip install geemap
%pip install scikit-learn
%pip install geopandas
%pip install mercantile
%pip install rasterio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 2.5 MB/s eta 0:00:0000:0100:01
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [rasterio]4/5 [rasterio]
Note: you may need to restart the kernel to use updated packages.


In [22]:
import geopandas as gpd
import pandas as pd

maus = gpd.read_file('Input/maus/Global_Mining_Polygons_v2.shp')
maus['bbox_xmin'] = maus.bounds['minx']
maus['bbox_xmax'] = maus.bounds['maxx']
maus['bbox_ymin'] = maus.bounds['miny']
maus['bbox_ymax'] = maus.bounds['maxy']

nicfi = gpd.read_file('Input/NICFIFootprint/NICFIFootprint.shp')

# Settings
zm = 13
sze = 640
min_overlap = 0.001

locations = pd.read_csv(f'Input/Boxes/box{zm}.csv')
locations = locations.assign(
    zoom=zm,
    size=sze,
    lat=locations['Lat'],
    lon=locations['Lon']
)[['lat', 'lon', 'size', 'zoom', 'tpe']]

print(locations.shape)


(25218, 5)


In [24]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import box
import mercantile
import rasterio
from rasterio.transform import from_bounds
import numpy as np

# Function to create a bounding box in lat/lon for a tile
def create_bbox(lat, lon, zoom, size):
    tile = mercantile.tile(lon, lat, zoom)
    bounds = mercantile.bounds(tile)
    return box(bounds.west, bounds.south, bounds.east, bounds.north)

# Convert each location to a polygon box
locations['geometry'] = locations.apply(lambda row: create_bbox(row['lat'], row['lon'], row['zoom'], row['size']), axis=1)
locations_gdf = gpd.GeoDataFrame(locations, geometry='geometry', crs='EPSG:4326')

# Reproject both GeoDataFrames to a projected CRS for accurate area calculation (e.g., EPSG:3857)
maus_proj = maus.to_crs(epsg=3857)
locations_proj = locations_gdf.to_crs(epsg=3857)

# Spatial join: find intersections
intersections = gpd.overlay(locations_proj, maus_proj, how='intersection')

# Calculate overlap area
intersections['intersection_area'] = intersections.geometry.area

# Calculate original location box area
locations_proj['box_area'] = locations_proj.geometry.area

# Merge intersection back with original location to get box area
intersections = intersections.merge(
    locations_proj[['lat', 'lon', 'box_area']],
    on=['lat', 'lon']
)

# Compute overlap fraction
intersections['overlap_fraction'] = intersections['intersection_area'] / intersections['box_area']

# Filter by minimum overlap threshold
valid_locations = intersections[intersections['overlap_fraction'] >= min_overlap]

# Drop duplicates in case of multiple maus overlaps
valid_locations = valid_locations.drop_duplicates(subset=['lat', 'lon'])

# Join back with original locations to get full data (if needed)
final_locations = locations.merge(valid_locations[['lat', 'lon']], on=['lat', 'lon'])

# final_locations is the filtered DataFrame
print(final_locations)

(5010, 6)
            lat        lon  size  zoom  tpe  \
0     45.010116   0.000000   640    13  pos   
1     14.377553   0.098815   640    13  pos   
2     59.931089   9.930875   640    13  pos   
3     17.934866  98.962903   640    13  pos   
4      1.531628  99.061718   640    13  pos   
...         ...        ...   ...   ...  ...   
5005  28.705693 -98.419423   640    13  pos   
5006  20.009995 -98.617052   640    13  pos   
5007  20.108810 -98.617052   640    13  pos   
5008  29.594998 -98.567645   640    13  pos   
5009  54.891545 -98.666459   640    13  pos   

                                               geometry  
0     POLYGON ((0.0439453125 44.99588261816545, 0.04...  
1     POLYGON ((0.1318359375 14.34954783718537, 0.13...  
2     POLYGON ((9.931640625 59.91097597079679, 9.931...  
3     POLYGON ((98.96484375 17.89511430374914, 98.96...  
4     POLYGON ((99.0966796875 1.4939713066293199, 99...  
...                                                 ...  
5005  POLYGON ((-98

/tmp/ipykernel_19647/3917669341.py:24: UserWarning: `keep_geom_type=True` in overlay resulted in 1 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersections = gpd.overlay(locations_proj, maus_proj, how='intersection')
